# EKG fra celler til hjertevektoren

In [ ]:
# 0) Opsætning — imports og plotindstillinger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Fra transmembranpotentiale til et EKG‑lignende signal

- Hver myocytes **transmembranpotentiale** $V_m(t)$ ændrer sig under depolarisering og repolarisering.
- Når mange celler aktiveres **sekventielt** (en udbredende bølgefront), summerer deres **transmembrane strømme** (tilnærmet relateret til $\mathrm{d}V_m/\mathrm{d}t$) til et **fjernfeltssignal** målt ved elektroder.
- I et elektrisk felt er EKG fra en fiber omtrent proportional med den **vægtede sum af $\mathrm{d}V_m/\mathrm{d}t$** på tværs af celler, skaleret af geometri og afledningsretning.

Vi simulerer: (1) et glat AP, (2) en 1D række celler med konstant udbredelseshastighed, (3) et EKG‑lignende signal som summen af $\mathrm{d}V_m/\mathrm{d}t$.


In [ ]:
from matplotlib.axes import Axes


# 1.1 Definér et simpelt aktionspotentiale og visualisér
def sigmoid(x, tau):
    return 1.0 / (1.0 + np.exp(-x / tau))


def simpelt_AP(
    t, t0=0.0, APD=300.0, tau_op=1.5, tau_rep=12.0, V_hvile=-85.0, V_top=30.0
):
    """
    Glat 'rektangulært' AP: V_hvile -> V_top -> V_hvile
    t i ms. APD ~ varighed af depolariseret fase.
    """
    op = sigmoid(t - t0, tau_op)  # glat depolarisering efter t0
    ned = sigmoid((t0 + APD) - t, tau_rep)  # ~1 før APD, ~0 efter APD
    boks = op * ned
    return V_hvile + (V_top - V_hvile) * boks


# Demo: enkelt AP
t = np.linspace(-50, 450, 2000)  # ms
Vm = simpelt_AP(t, t0=0, APD=280, tau_op=2.0, tau_rep=10.0)

fig, axs = plt.subplots(1, 2, figsize=(12, 5))
axs: list[Axes]
axs[0].plot(t, Vm)
axs[0].set_title("Enkeltcelle-aktionspotentiale (simpel model)")
axs[0].set_xlabel("Tid (ms)")
axs[0].set_ylabel("V_m (mV)")

# Tidsderivativ (tilnærmet transmembran strømsignatur)
dVm_dt = np.gradient(Vm, t[1] - t[0])
axs[1].plot(t, dVm_dt, color="C1")
axs[1].set_title("Estimat af transmembran strøm ~ dV_m/dt")
axs[1].set_xlabel("Tid (ms)")
axs[1].set_ylabel("dV_m/dt (mV/ms)")
plt.show()

In [ ]:
# 1.2 Udbredelse langs en 1D fiber og et EKG‑lignende signal
v = 0.6  # mm/ms ledningshastighed (0.6 mm/ms = 0.6 m/s)
APD = 280
afledningsretning_grader = 60.0
theta_grader = 30.0  # fiber-/hjerteakse‑orientering (frontalplan)


N = 200  # antal celler
dx = 0.1  # mm pr. celle (vilkårligt)

t_ms = np.linspace(0, 350, 2000)  # simuleringstid (ms)
dt = t_ms[1] - t_ms[0]
x = np.arange(N) * dx
t_akt = x / v  # aktiveringstid for hver celle (ms)


def enhedsvektor(grader):
    rad = np.deg2rad(grader)
    return np.array([np.cos(rad), np.sin(rad)])


e_hjerte = enhedsvektor(theta_grader)
e_afl = enhedsvektor(afledningsretning_grader)
orientation_weight = float(np.dot(e_afl, e_hjerte))  # i [-1,1]

Vm_alle = np.zeros((len(t_ms), N))
for i in range(N):
    Vm_alle[:, i] = simpelt_AP(t_ms, t0=t_akt[i], APD=APD, tau_op=2.0, tau_rep=10.0)

dVm_dt_alle = np.gradient(Vm_alle, dt, axis=0)
ekg_lign = orientation_weight * dVm_dt_alle.sum(axis=1)

fig, axs = plt.subplots(1, 2, figsize=(10, 5), sharex=True)
from matplotlib.axes import Axes

axs: list[Axes]
axs[0].plot(t_ms, Vm_alle[:, 0], label="Celle 0")
axs[0].plot(t_ms, Vm_alle[:, int(N / 2)], label=f"Celle {int(N/2)}", alpha=0.8)
axs[0].plot(t_ms, Vm_alle[:, -1], label=f"Celle {N-1}", alpha=0.7)
axs[0].set_ylabel("V_m (mV)")
axs[0].set_title("Udvalgte celler med forsinkelse pga. udbredelse")
axs[0].legend(loc="best")

axs[1].plot(t_ms, ekg_lign, color="C3")
axs[1].set_xlabel("Tid (ms)")
axs[1].set_ylabel("Afledningssignal (arb. enheder)")
axs[1].set_title("EKG-lignende signal fra udbredende aktivering")
plt.tight_layout()
plt.show()

print(f"Orienteringskobling (afledning vs. fiber): {orientation_weight:.2f}")

1. OPGAVE A: Ændrer v (ledningshastighed) og observer QRS-bredde.
2. OPGAVE B: Ændrer APD og observer ændringer i T-lignende komponent.
3. OPGAVE C: Sæt afledningsretning tæt på vinkelret på theta_grader og se amplitudefald.


*Noter: I virkeligheden har vi flere forskellige typer af celler i hjertet, som alle har unikke depolariserings mønstre, se Kapitel `5.5`. Et ekempel på dette kan ses i billedet herunder. Men princippet forbliver det samme.*



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the image
img = mpimg.imread("files/Shapes_of_the_cardiac_action_potential_in_the_heart.jpg")

# Display the image
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis("off")  # Hide axes
plt.title("Cardiac Action Potential Shapes")
plt.show()

## Rekonstruér hjertevektor HV fra Afledning I & II

Lad os nu arbejde lidt med nogle specifikke værdier fra et EKG, for at blive fortrolige med dem. 

Specifikt, lad os fokuserer på en enkelt måling fra et EKG, hvor vi målte værdierne: Lead III:`0.5mV` og Lead II: `0.30mV`.

Der er flere måder at løse den på. Bogen har `Example 5.6.1`, som bestemt kan bruges. 
I filen `files/Hjertevektor_Ex_5_6_1.pdf` kan løsningen til samme opgave ses, men med udgangspunkt i Linear Algebra.
Når det kommer til generaliseringer vil jeg personligt mene at dette er en mere overskuelig måde at løse denne type opgaver på.

1. Når man skal udregne Lead I, II, III, når elektroderne er placeret på hhv: ventre arm (LA), højre arm (RA), venstre ben (LL), og højre ben (RL)?
2. Hvad ville hjertevektoren på dette tidspunkt være for denne person (Lead III:`0.5mV` og Lead II: `0.30mV`)?
    *Hint1: Svaret er $HV_x=2.57mV$, $HV_y=-0.36mV$*
3. Det er din opgave at løse opgaven med Lead I:`0.5mV` og Lead II: `0.30mV`. Er et elektrisk udslag på omkring egentlig 1mV realistisk?
4. Løs opgaven i Python.



In [ ]:
# Solve

## Plot **målt EKG** fra **2 RR‑cyklusser** samt Hjertevektoren
I denne del bruger vi en fil `ecg_data1.csv` (kolonner: `time_s, I, II, III`).
Målet er at: 
1. Indlæse 10 s data
2. Vælge **start** og **stop** for  **2 RR‑cyklusser**
3. Plotte udsnittet pænt.
4. Omregn enten Lead I og Lead II, eller Lead I og Lead III til en hjertevektor, og plot den.

In [ ]:
#

In [ ]:
# 5.1 Dette script indlæser dataset og finder sampling rate for dig.
from pathlib import Path

csv_path = "files/mhd-effect-ecg-mri/ecg_data1.csv"  # tilpas sti hvis nødvendigt (burde det ikke være)
df: pd.DataFrame = pd.read_csv(Path(csv_path))

fs = 1.0 / df["time_s"].diff().median()  # Sampling frekvens

duration = float(len(df) / fs)  # duration
print(f"CSV: Estimeret fs = {fs:.2f} Hz | Varighed = {duration:.2f} s | N = {len(df)}")
df.head()

In [ ]:
from scipy.signal import butter, sosfilt


# 5.2 Hjælpefunktioner: fitler og plot af signalerne
def butter_bandpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = np.array(cutoff) / nyq
    sos = butter(order, normal_cutoff, output="sos", btype="bandpass")
    return sos


def butter_bandpass_filter(data, cutoff, fs, order=2) -> np.ndarray:
    data = np.array(data)

    sos = butter_bandpass(cutoff, fs, order=order)
    data = data - data[0]
    y = sosfilt(sos, data)

    return np.array(y)


def plot_ecg_segment(
    df_seg: pd.DataFrame,
    *,
    y_columns: list[str],
    x_column: str = "time_s",
    use_baseline_correction=True,
    fs: float | int | None = None,
    title="EKG-udsnit",
):
    assert isinstance(y_columns, list | tuple | set)
    assert set(y_columns).issubset(df_seg.columns)

    t = df_seg[x_column].to_numpy()

    ylabel = "Amplitude (a.e.)"
    fig, ax = plt.subplots(figsize=(10, 5))
    for lbl in y_columns:
        y0 = df_seg[lbl].to_numpy()
        if use_baseline_correction and fs is not None:
            y = butter_bandpass_filter(y0, [1.0, 100.0], fs)
        else:
            y = y0
        ax.plot(t, y, label=lbl)

    ax.set_xlabel("Tid (s)")
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend(loc="upper right")
    plt.tight_layout()
    plt.show()

In [ ]:
# Vælg start/stop for  1 RR‑cyklus og plot udsnittet, undgå steder med støj!
# >>> UDFYLD HER: tider i sekunder (brug fuld længde-plottet som guide)
t_start = 0  # <-- Angiv starttid (s)
t_stop = 1000  # <-- Angiv sluttid (s) så udsnittet dækker PRÆCIS 2 RR

mask = (df["time_s"] >= t_start) & (df["time_s"] <= t_stop)
df_seg = df.loc[mask].copy()
if df_seg.empty:
    raise ValueError("Udsnittet er tomt. Tjek t_start/t_stop.")
start_t = float(df_seg["time_s"].iloc[0])
stop_t = float(df_seg["time_s"].iloc[-1])
dur_t = stop_t - start_t
print(f"Udsnit: {start_t:.3f}s → {stop_t:.3f}s ({dur_t:.3f}s), N={len(df_seg)}")
plot_ecg_segment(
    df_seg,
    y_columns=["I", "III"],
    use_baseline_correction=False,
    fs=float(fs),
    title=f"CSV: 1 RR-cyklus ({t_start:.2f}-{t_stop:.2f} s)",
)

### Plot Hjertevektoren

I bogen finde der en `Figure 5.6.8`, som viser hvordan hjertevektoren ser ud for en enkelt hjertecyklus. 
Dette er naturligvis en model, og er ikke altid hvordan HV ser ud. 
Det er din opgave at lave en HV plot, for det EKG du lige har plottet ovenover, ved at benytte den algoritme du har udnyttet ovenover.

Spørgsmål:
1. Hvad sker der med hjertevektor plottet, hvis du filterer dine leads med et butterwork filter, i.e. `LeadI = butter_bandpass_filter(df_seg["I"], [1, 100], fs)`?
2. Er der forskel på om man benytter Lead I+II / I+II / II+III til at beregne HV?

In [ ]:
import numpy as np
import math
from matplotlib.axes import Axes

leadI = df_seg["I"]  # butter_bandpass_filter(df_seg["I"], [1, 100], fs)
leadII = df_seg["II"]  # butter_bandpass_filter(df_seg["II"], [1, 100], fs)
leadIII = df_seg["III"]  # butter_bandpass_filter(df_seg["III"], [1, 100], fs)


# Plot hjertevektoren

## Opsamling / Refleksion
1. **Celler → EKG**: Hvilke aspekter af AP‑formen (fx APD) og udbredelseshastigheden påvirkede signalet mest?
2. **Hjertevektor → Afledninger**: Hvordan ændrer retningen af $\mathbf{HV}$ de relative amplituder i I, II, III?
3. **Einthovens lov**: Hvorfor gælder II = I + III (hint: afledninger er differenser af elektrodepotentialer)?
4. **Invers mapping**: Hvilke antagelser ligger bag rekonstruktionen af $\mathbf{HV}$ fra I & II?
